In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
Cambridge = 42.3731956,-71.1198561

In [3]:
sub_CA = pd.read_csv("../Data/sub_CA.csv")


# Cambridge (USA)

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about

In [6]:
starbucks_CA = fun.get_results("starbucks", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results_first(starbucks_CA, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about

In [7]:
kinder_CA = fun.get_results("preschool", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(kinder_CA, df_CA, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [8]:
club_CA = fun.get_results("night club", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(club_CA, df_CA, "Club")

- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [9]:
vegan_CA = fun.get_results("vegan", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(vegan_CA, df_CA, "Vegan restaurant")

- We look for basketball arena (Chase Center) around the office and clean the results keeping only the columns that we are interested about

In [10]:
Basketball_CA = fun.get_results("TD Garden", Cambridge, 1, 5000, token_foursquare)
df_CA = fun.clean_results(Basketball_CA, df_CA, "Basketball arena")

- We look for dog hairdressers around the office and clean the results keeping only the columns that we are interested about

In [11]:
pet_CA = fun.get_results("dog grooming", Cambridge, 5, 3000, token_foursquare)
df_CA = fun.clean_results_lo(pet_CA, df_CA, "Pet hairdresser")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [12]:
Airports_CA = fun.get_results_airport("Airport", Cambridge, 2, 30000, token_foursquare)
df_CA = fun.clean_results_lo(Airports_CA, df_CA, "Airport")

- We add the data of the new office that we have selected

In [13]:
df_CA = fun.add_company(df_CA, "5, John F. Kennedy St, Cambridge, MA 02138", "West Cambridge", 42.3731956,-71.1198561)

- We create a new row with the distance from the office to the diferent places

In [14]:
df_CA["Distance"] = df_CA.apply(fun.calc_distance_CA, axis = 1).round(2)

In [15]:
df_CA

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"5, John F. Kennedy St, Cambridge, MA 02138",West Cambridge,42.373196,-71.119856,NaN,0.00
1,Starbucks,"1662 Massachusetts Ave (at Shepard St), Cambri...",Central Square,42.382029,-71.119939,Coffee,981.24
2,Starbucks,"468 Broadway (at Broadway Marketplace), Cambri...",Central Square,42.373858,-71.112954,Coffee,573.23
3,Starbucks,"711 Somerville Ave (at Elm St), Somerville, MA...",Central Square,42.386097,-71.114168,Coffee,1507.72
4,Starbucks,"217 Western Ave, Boston, MA 02134",Allston,42.363412,-71.130402,Coffee,1391.28
5,Starbucks,"750 Memorial Dr, Cambridge, MA 02139",Allston,42.358000,-71.115075,Coffee,1733.27
6,Starbucks,"370 Western Ave, Brighton, MA 02135",North Brighton,42.360988,-71.137756,Coffee,2003.21
7,Starbucks,"699 Mount Auburn St, Cambridge, MA 02138",Larchwood,42.375352,-71.148982,Coffee,2410.84
8,Starbucks,"1065 Commonwealth Ave, Boston, MA 02215",Allston,42.352500,-71.123479,Coffee,2318.17
9,Starbucks,"1304 Commonwealth Ave, Boston, MA 02134",Allston-Brighton,42.348482,-71.134065,Coffee,2984.34


In [16]:
df_CA.to_csv("../data/df_CA.csv", index=False)

In [17]:
CA_map = Map(location = Cambridge, zoom_start = 12)

In [18]:
for index, row in df_CA.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["category"] == "Coffee":        
        icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="graduation-cap",
            icon_color = "yellow"
        )
    elif row["category"] == "Club":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "white"
        )
    elif row["category"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="cutlery",
            icon_color = "white"
        )
    elif row["category"] == "Basketball arena":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="futbol-o",
            icon_color = "white"
        )
    elif row["category"] == "Airport":
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="plane",
            icon_color = "white"
            
        )
    elif row["category"] == "Pet hairdresser":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="scissors",
            icon_color = "white"
        )
    else:
        row["category"] == "Company",
        icon = Icon (
        color="black",
        opacity = 0.9,
        prefix = "fa",
        icon = "briefcase",
        icon_color = "white",
        icon_size=(30, 30)
        )
            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(CA_map)

In [19]:
for index, row in sub_CA.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["Lat"], row["Long"]], "tooltip": row["name"]}
    
    #2. Icon       
    icon = Icon (
    color="white",
    opacity = 0.6,
    prefix = "fa",
    icon="building",
    icon_color = "black"
        )

            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(CA_map)

In [20]:
CA_map

In [21]:
CA_map.save('../images/Cambridge_map.html')